### Energy Imports & Exports

In [20]:
import pandas as pd
import re

# source: generation Mix (https://www.carbonfootprint.com/docs/2020_09_emissions_factors_sources_for_2020_electricity_v14.pdf)
carbon = {
    "AT": 114, "BE": 150, "CZ": 401, "DK": 114, "FR": 42, "DE": 332,
    "IE": 256, "IT": 285, "LU": 124, "NL": 253, "NO": 29, "PL": 592,
    "ES": 153, "SE": 35, "CH": 33, "GB": 217
}

name = "den_2025"
exporting_country = "DK"
df = pd.read_csv(f"energy_imports_exports/den/{name}.csv")  # uncomment if needed

export_factor = carbon.get(exporting_country)
if export_factor is None:
    raise ValueError(f"No carbon factor found for exporting_country='{exporting_country}'")

# --- 1) Extract hour start datetime from MTU (left side of " - ")
df["datetime"] = pd.to_datetime(
    df["MTU"].str.split(" - ").str[0],
    dayfirst=True,  # MTU looks like dd/mm/yyyy in your screenshot
    errors="coerce"
)

# --- 2) Extract country codes like "Austria (AT)" -> "AT"
code_re = r"\(([A-Z]{2})\)$"
df["out_code"] = df["Out Area"].str.extract(code_re)[0]
df["in_code"]  = df["In Area"].str.extract(code_re)[0]

# --- 3) Flow column (ensure numeric)
df["flow"] = pd.to_numeric(df["Physical Flow (MW)"], errors="coerce").fillna(0.0)

# --- 4) Classify direction relative to Germany
is_export = (df["out_code"] == exporting_country) & (df["in_code"] != exporting_country)
is_import = (df["in_code"] == exporting_country) & (df["out_code"] != exporting_country)

# --- 5) Row-wise energy & carbon
df["energy_export"] = df["flow"].where(is_export, 0.0)
df["energy_import"] = df["flow"].where(is_import, 0.0)

# Export carbon uses the exporting countries carbon factor (e.g for germany: 401.53 * 332)

df["carbon_export"] = df["energy_export"] * carbon[exporting_country]

# Import carbon uses exporting country's factor (out_code)
df["carbon_import"] = df["energy_import"] * df["out_code"].map(carbon).fillna(0.0)

# --- 6) Aggregate to hourly totals
out = (
    df.groupby("datetime", as_index=False)[
        ["energy_import", "carbon_import", "energy_export", "carbon_export"]
    ]
    .sum()
    .rename(columns={
        "energy_import": "total_energy_import",
        "carbon_import": "total_carbon_import",
        "energy_export": "total_energy_export",
        "carbon_export": "total_carbon_export",
    })
    .sort_values("datetime")
)

# fill missing values with zeros as no energy imported or exported
full_hours = pd.date_range(out["datetime"].min(), out["datetime"].max(), freq="h")
out = (
    out.set_index("datetime")
       .reindex(full_hours, fill_value=0)
       .rename_axis("datetime")
       .reset_index()
)

out.to_csv(f"energy_imports_exports/den/{name}_ci.csv", index=False)